In [1]:
import sys,os
sys.path.insert(0, f"{os.path.dirname(os.path.realpath('__file__'))}/../../")
from rxnrecer.config import config as cfg
from rxnrecer.utils import file_utils as futils
from rxnrecer.utils import bio_utils as butils
from rxnrecer.utils import format_utils as format_utils
from rxnrecer.lib.llm import qa as llm_qa
from rxnrecer.cli import predict as predictor
%load_ext autoreload
%autoreload 2

In [2]:
res_rxnrecer_s1 = predictor.step_by_step_prediction(
    input_data=f'{cfg.SAMPLE_DIR}sample10.fasta', 
    output_file=None, 
    output_format='tsv', 
    mode='s1',
    batch_size=100
)
res_rxnrecer_s1.head(3)

Detected input is a FASTA file: /hpcfs/fhome/shizhenkun/codebase/rxnrecer_production/data/sample/sample10.fasta
Step 1: Preparing input data, loading 10 proteins
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 10 proteins


  0%|          | 0/1 [00:00<?, ?it/s]

Running RXNRECer-S1 ...


Predicting reactions: 100%|██████████| 5/5 [00:00<00:00,  5.77it/s]


INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 1/1 [00:03<00:00,  3.24s/it]


,input_id,RXNRECer,RXNRECer_with_prob,rxn_details
0,A0A7H0DNE2,RHEA:14709;RHEA:24076,"{'RHEA:14709': 0.997943, 'RHEA:24076': 0.994297}","[{'reaction_id': 'RHEA:14709', 'prediction_con..."
1,C0HK73,-,{'-': 0.999998},"[{'reaction_id': '-', 'prediction_confidence':..."
2,F2XF94,RHEA:32539,{'RHEA:32539': 0.955101},"[{'reaction_id': 'RHEA:32539', 'prediction_con..."


In [3]:
res_rxnrecer_s2 = predictor.step_by_step_prediction(
    input_data=f'{cfg.SAMPLE_DIR}sample10.fasta', 
    output_file=None, 
    output_format='tsv', 
    mode='s2',
    batch_size=100
)
res_rxnrecer_s2.head(3)

Detected input is a FASTA file: /hpcfs/fhome/shizhenkun/codebase/rxnrecer_production/data/sample/sample10.fasta
Step 1: Preparing input data, loading 10 proteins
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 10 proteins


  0%|          | 0/1 [00:00<?, ?it/s]

Running RXNRECer-S1 ...


Predicting reactions: 100%|██████████| 5/5 [00:00<00:00,  8.57it/s]


Running RXNRECer-S2 ...
Transferred model to GPU


Processing Sequences: 100%|██████████| 10/10 [00:00<00:00, 48.96it/s]


Processed 10 proteins in 0.21 seconds
INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 1/1 [01:13<00:00, 73.58s/it]


,input_id,RXNRECer,RXNRECer_with_prob,rxn_details
0,A0A7H0DNE2,RHEA:14709;RHEA:24076;RHEA:32187,"{'RHEA:14709': 0.999931, 'RHEA:24076': 0.99993...","[{'reaction_id': 'RHEA:14709', 'prediction_con..."
2,C0HK73,-,{'-': 0.999998},"[{'reaction_id': '-', 'prediction_confidence':..."
3,F2XF94,RHEA:32539,{'RHEA:32539': 0.999942},"[{'reaction_id': 'RHEA:32539', 'prediction_con..."


In [6]:
cfg.LLM_API_KEY = 'your_api_key_here'
cfg.LLM_API_URL = "https://openrouter.ai/api/v1"

res_rxnrecer_s3 = predictor.step_by_step_prediction(
    input_data=f'{cfg.SAMPLE_DIR}sample10.fasta', 
    output_file=None, 
    output_format='tsv', 
    mode='s3',
    batch_size=100
)
res_rxnrecer_s3.head(3)

Detected input is a FASTA file: /hpcfs/fhome/shizhenkun/codebase/rxnrecer_production/data/sample/sample10.fasta
Step 1: Preparing input data, loading 10 proteins
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 10 proteins


  0%|          | 0/1 [00:00<?, ?it/s]

Running RXNRECer-S1 ...


Predicting reactions: 100%|██████████| 5/5 [00:00<00:00,  8.50it/s]


Running RXNRECer-S3 ...
Transferred model to GPU


Processing Sequences: 100%|██████████| 10/10 [00:00<00:00, 67.47it/s]


Processed 10 proteins in 0.15 seconds
INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 1/1 [02:04<00:00, 124.29s/it]


,input_id,RXNRECer,RXNRECer_with_prob,rxn_details
0,A0A7H0DNE2,RHEA:14709;RHEA:24076;RHEA:32187,"{'RHEA:14709': 0.999931, 'RHEA:24076': 0.99993...","[{'reaction_id': 'RHEA:14709', 'prediction_con..."
2,C0HK73,-,{'-': 0.999998},"[{'reaction_id': '-', 'prediction_confidence':..."
3,F2XF94,RHEA:32539,{'RHEA:32539': 0.999942},"[{'reaction_id': 'RHEA:32539', 'prediction_con..."


In [5]:
# res_rxnrecer_s3.to_csv(f'{cfg.TEMP_DIR}/aa.tsv', index=False, sep='\t', float_format="%.6f")
# predictor.save_data(res_rxnrecer_s3, f'{cfg.TEMP_DIR}/aa.json', output_format='json')